# Intro Tutorial on Relation Extraction from News Articles

## Part III: Creating or Loading Evaluation Labels

In [ ]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

## Part III(a): Creating Evaluation Labels in the `Viewer`

We repeat our definition of the `Spouse` `Candidate` subclass from Part II.

In [ ]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

## Loading the development `CandidateSet`

We will start by viewing the development `CandidateSet` we created in Part II in the `Viewer`.

First we reload the development `CandidateSet`.

In [ ]:
from snorkel.models import CandidateSet

cs = session.query(CandidateSet).filter(CandidateSet.name == 'News Development Candidates').one()
cs

## Labeling the `CandidateSet` in the `Viewer`

We create a `Viewer` to annotate them manually.

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(cs[:300], session, annotator_name="Tutorial Part III User")
else:
    sv = None

We now open the Viewer.

You can mark each `Candidate` as true or false. Remember that <span style="color:red">red</span> denotes the first argument (chemical) and <span style="color:blue">blue</span> denotes the second (disease). Try it!

These labels are automatically saved in the database backend, and can be accessed using the annotator's name ('Tutorial Part III User') as the AnnotationKey.

In [ ]:
sv

## Part III(b): Loading External Evaluation Labels

Loading in external annotations can be a bit messier, since these external annotations could be in any format.  Here, we'll provide an example of how to use the `ExternalAnnotationsLoader` helper class to make this a bit simpler.

In [ ]:
from snorkel.loaders import ExternalAnnotationsLoader

loader = ExternalAnnotationsLoader(session, Disease, 
                                   'News Development Candidates',
                                   'News Development Labels -- Gold')

Now we'll load the rest of the annotations:

In [ ]:
for set_name in ['Training', 'Test']:
    loader = ExternalAnnotationsLoader(session, Disease, 
                                       'News %s Candidates -- Gold' % set_name,
                                       'News %s Labels -- Gold' % set_name)
    print session.query(Label).filter(Label.key == loader.annotation_key).count()